In [8]:
import pandas as pd
import numpy as np
import os

# ============================================================
# Load CSVs
# ============================================================
constructs   = pd.read_csv("constructs.csv")
dimensions   = pd.read_csv("dimensions.csv")
questions    = pd.read_csv("questions.csv")
fact         = pd.read_csv("AI_CapScan_3NF_fact_table.csv")
respondents  = pd.read_csv("AI_CapScan_3NF_ResponseId.csv")

# ============================================================
# Prepare Construct–Dimension Mapping
# ============================================================
construct_map = constructs.merge(dimensions, on="dimension_id", how="left")

construct_map["column"] = [
    f"{row['dimension_id']}_{(row['construct_id'] - 1) % 4 + 1}_1"
    for _, row in construct_map.iterrows()
]

# ============================================================
# Extract Respondent + Score Columns
# ============================================================
score_cols = [c for c in fact.columns if "_" in c and c.count("_") == 2]
df_scores = fact[["ResponseId_id"] + score_cols]

# ============================================================
# Reshape Wide → Long
# ============================================================
long_df = df_scores.melt(
    id_vars=["ResponseId_id"],
    var_name="column",
    value_name="score"
)

# ============================================================
# Merge with Construct Metadata
# ============================================================
merged = long_df.merge(construct_map, on="column", how="left")

# ============================================================
# Add English Synthetic Filters (Realistic Geography)
# ============================================================

industries_en = [
    "Financial Services", "Healthcare", "Manufacturing",
    "Education", "Technology", "Government"
]

countries_en = [
    "Netherlands", "Germany", "France", "United Kingdom",
    "United States", "India"
]

continents_en = [
    "Europe", "North America", "Asia", "South America", "Africa"
]

roles_en = [
    "Leadership", "Staff", "IT & Data", "Strategy & Policy"
]

def deterministic_choice(id_value, options):
    np.random.seed(abs(hash(str(id_value))) % (2**32))
    return np.random.choice(options)

merged["industry_synthetic"]  = merged["ResponseId_id"].apply(lambda x: deterministic_choice(x, industries_en))
merged["country_synthetic"]   = merged["ResponseId_id"].apply(lambda x: deterministic_choice(x, countries_en))
merged["continent_synthetic"] = merged["ResponseId_id"].apply(lambda x: deterministic_choice(x, continents_en))
merged["role_synthetic"]      = merged["ResponseId_id"].apply(lambda x: deterministic_choice(x, roles_en))

# ============================================================
# Clean & Rename Columns (Frontend Schema)
# ============================================================
demo = merged.rename(columns={
    "dimension_name": "dimension",
    "construct_name": "construct"
})[[
    "ResponseId_id", "dimension_id", "dimension",
    "construct_id", "construct", "score",
    "industry_synthetic", "country_synthetic",
    "continent_synthetic", "role_synthetic"
]]

# ============================================================
# Save Final Dataset
# ============================================================
out_path = "capability_demo.csv"
demo.to_csv(out_path, index=False)

demo.head(10)

,ResponseId_id,dimension_id,dimension,construct_id,construct,score,industry_synthetic,country_synthetic,continent_synthetic,role_synthetic
0,0046a5e5,1,Strategy and Vision,1,Alignment with Business Goals,4.0,Government,India,North America,Staff
1,a2f7ce34,1,Strategy and Vision,1,Alignment with Business Goals,2.0,Technology,United States,Africa,Leadership
2,113b34f0,1,Strategy and Vision,1,Alignment with Business Goals,1.0,Manufacturing,France,Asia,IT & Data
3,d3de8ea9,1,Strategy and Vision,1,Alignment with Business Goals,7.0,Government,India,Africa,Staff
4,26d0d233,1,Strategy and Vision,1,Alignment with Business Goals,3.0,Government,India,North America,Strategy & Policy
5,cc2a2a05,1,Strategy and Vision,1,Alignment with Business Goals,2.0,Technology,United States,Africa,Leadership
6,98a96934,1,Strategy and Vision,1,Alignment with Business Goals,2.0,Education,United Kingdom,South America,Strategy & Policy
7,2c7b1ffe,1,Strategy and Vision,1,Alignment with Business Goals,3.0,Financial Services,Netherlands,Europe,IT & Data
8,a1f58ce0,1,Strategy and Vision,1,Alignment with Business Goals,3.0,Manufacturing,France,Asia,IT & Data
9,75bec41c,1,Strategy and Vision,1,Alignment with Business Goals,7.0,Financial Services,Netherlands,Europe,Leadership
